# **<p style="text-align: center;">Aprendizaje por refuerzo - Ingeniería en Inteligencia Artificial</p>**
## **<p style="text-align: center;">Trabajo práctico Final - SAC</p>**

#### <p style="text-align: center;">Pettinari Fausto, Schuemer Ignacio, Torres Santiago </p>
#### <p style="text-align: center;">Profesores: Claudio Pose, Gabriel Torre, Nicolás Romero, Tomás Chimenti</p>

# **Donkey Car VAE Enviroment**

In [1]:
import sys
from pathlib import Path

# Add project roots to import path
REPO_ROOT = Path("..").resolve()
sys.path.append(str(REPO_ROOT))
sys.path.append(str(REPO_ROOT / "learning-to-drive-in-5-minutes"))


In [2]:
import sys
import numpy as np
import gymnasium as gym
import torch

# Alias gymnasium as gym for donkey_gym, which expects the old API
sys.modules.setdefault("gym", gym)
sys.modules.setdefault("gym.spaces", gym.spaces)
sys.modules.setdefault("gym.envs", gym.envs)
sys.modules.setdefault("gym.envs.registration", gym.envs.registration)
sys.modules.setdefault("gym.utils", gym.utils)


<module 'gymnasium.utils' from '/home/san/miniconda3/lib/python3.12/site-packages/gymnasium/utils/__init__.py'>

In [5]:
import io
import zipfile
from typing import Any
from pathlib import Path
import numpy as np
import gymnasium as gym
import torch
import cv2
from gymnasium.envs.registration import EnvSpec
from donkey_gym.envs.vae_env import DonkeyVAEEnv
from config import LEVEL, FRAME_SKIP, MIN_THROTTLE, MAX_THROTTLE, MAX_CTE_ERROR, ROI
from vae.data_loader import preprocess_input

MAX_EPISODE_STEPS = 1000  # fallback horizon for wrappers expecting a finite episode length
vae_archive_dir = REPO_ROOT / "learning-to-drive-in-5-minutes" / "ae-32_mountain" / "archive"
print(f"Loading VAE weights from extracted archive dir: {vae_archive_dir}")
assert vae_archive_dir.exists(), "VAE archive folder not found."

import os
os.environ["DONKEY_SIM_PATH"] = "/home/san/Documents/Ingenieria UdeSA/RL/DonkeySimLinux/donkey_sim.x86_64"
os.environ["DONKEY_SIM_PORT"] = "9090"  # match the sim’s default
# optional: os.environ["DONKEY_SIM_HEADLESS"] = "1"



def _load_torch_vae_checkpoint(extracted_archive: Path):
    """Load a torch checkpoint from an extracted torch archive folder."""
    buffer = io.BytesIO()
    with zipfile.ZipFile(buffer, "w") as zf:
        for file_path in extracted_archive.rglob("*"):
            if file_path.is_file():
                # torch expects entries like archive/data.pkl, archive/data/0, etc.
                rel_path = file_path.relative_to(extracted_archive.parent)
                info = zipfile.ZipInfo(str(rel_path), date_time=(1980, 1, 1, 0, 0, 0))
                info.compress_type = zipfile.ZIP_DEFLATED
                zf.writestr(info, file_path.read_bytes())
    buffer.seek(0)
    try:
        checkpoint = torch.load(buffer, map_location="cpu", weights_only=False)
    except TypeError:
        checkpoint = torch.load(buffer, map_location="cpu")
    return checkpoint


class TorchConvEncoder(torch.nn.Module):
    def __init__(self, z_size: int):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Conv2d(3, 16, kernel_size=4, stride=2),
            torch.nn.ReLU(),
            torch.nn.Conv2d(16, 32, kernel_size=4, stride=2),
            torch.nn.ReLU(),
            torch.nn.Conv2d(32, 64, kernel_size=4, stride=2),
            torch.nn.ReLU(),
            torch.nn.Conv2d(64, 128, kernel_size=4, stride=2),
            torch.nn.ReLU(),
        )
        self.proj = torch.nn.Linear(128 * 3 * 8, z_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        h = self.encoder(x)
        h = h.reshape(h.size(0), -1)
        return self.proj(h)


class TorchVAEWrapper:
    """Minimal VAE wrapper that matches the Donkey env API (encode + z_size)."""

    def __init__(self, archive_dir: Path):
        checkpoint = _load_torch_vae_checkpoint(archive_dir)
        z_size = int(checkpoint["data"]["z_size"])
        self.model = TorchConvEncoder(z_size)
        self.model.load_state_dict(checkpoint["state_dict"], strict=False)
        self.model.eval()
        self.z_size = z_size
        self.normalization_mode = checkpoint["data"].get("normalization_mode", "rl")

    def encode(self, observation: Any):
        # If we somehow already receive a latent vector, pass it through
        obs_arr = np.asarray(observation)
        if obs_arr.ndim < 3 or obs_arr.shape[-1] != 3:
            return obs_arr.reshape(1, -1).astype(np.float32)

        h, w = obs_arr.shape[:2]
        # Crop if larger than ROI, otherwise resize up to the expected VAE input
        if h > ROI[3] and w > ROI[2]:
            cropped = obs_arr[int(ROI[1]): int(ROI[1] + ROI[3]), int(ROI[0]): int(ROI[0] + ROI[2])]
        else:
            cropped = cv2.resize(obs_arr, (ROI[2], ROI[3]), interpolation=cv2.INTER_AREA)

        processed = preprocess_input(cropped.astype(np.float32), mode=self.normalization_mode)
        tensor = torch.from_numpy(processed).permute(2, 0, 1).unsqueeze(0)
        with torch.no_grad():
            z = self.model(tensor)
        return z.cpu().numpy()


class GymnasiumDonkeyWrapper(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 30}

    def __init__(self, donkey_env: DonkeyVAEEnv):
        self.donkey_env = donkey_env
        self._step_count = 0
        obs_dim = int(np.prod(donkey_env.observation_space.shape))
        self.observation_space = gym.spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(obs_dim,),
            dtype=np.float32,
        )
        self.action_space = gym.spaces.Box(
            low=donkey_env.action_space.low.astype(np.float32),
            high=donkey_env.action_space.high.astype(np.float32),
            dtype=np.float32,
        )
        self.spec = EnvSpec(id="DonkeyVae-v0", max_episode_steps=MAX_EPISODE_STEPS)
        self.render_mode = "human"
        self.max_episode_steps = MAX_EPISODE_STEPS

    @staticmethod
    def _flatten_obs(obs: Any) -> np.ndarray:
        return np.asarray(obs, dtype=np.float32).reshape(-1)

    def reset(self, seed=None, options=None):
        self._step_count = 0
        if seed is not None:
            self.donkey_env.seed(seed)
        obs = self.donkey_env.reset()
        return self._flatten_obs(obs), {}

    def step(self, action):
        obs, reward, done, info = self.donkey_env.step(action)
        self._step_count += 1
        terminated = bool(done)
        truncated = self._step_count >= self.max_episode_steps
        obs = self._flatten_obs(obs)
        info = info or {}
        if truncated:
            info.setdefault("TimeLimit.truncated", True)
        return obs, reward, terminated, truncated, info

    def render(self, mode="human"):
        return self.donkey_env.render(mode=mode)

    def close(self):
        try:
            self.donkey_env.close_connection()
        finally:
            try:
                self.donkey_env.exit_scene()
            except Exception:
                pass


def make_donkey_vae_env():
    vae = TorchVAEWrapper(vae_archive_dir)
    donkey_env = DonkeyVAEEnv(
        level=LEVEL,
        frame_skip=FRAME_SKIP,
        vae=vae,
        const_throttle=None,
        min_throttle=MIN_THROTTLE,
        max_throttle=MAX_THROTTLE,
        max_cte_error=MAX_CTE_ERROR,
        n_command_history=0,
    )
    return GymnasiumDonkeyWrapper(donkey_env)


env = make_donkey_vae_env()
print(f"Env ready with obs shape {env.observation_space.shape} and action shape {env.action_space.shape}")


Loading VAE weights from extracted archive dir: /home/san/Documents/Ingenieria UdeSA/RL/RL-SAC/learning-to-drive-in-5-minutes/ae-32_mountain/archive
Starting DonkeyGym env
Donkey subprocess started
Waiting for sim to start...if the simulation is running, press EXIT to go back to the menu
Waiting for sim to start...if the simulation is running, press EXIT to go back to the menu
Waiting for sim to start...if the simulation is running, press EXIT to go back to the menu
Waiting for sim to start...if the simulation is running, press EXIT to go back to the menu
Waiting for sim to start...if the simulation is running, press EXIT to go back to the menu
Waiting for sim to start...if the simulation is running, press EXIT to go back to the menu
Waiting for sim to start...if the simulation is running, press EXIT to go back to the menu


KeyboardInterrupt: 

#### **Logging & Debugging (Random Agent)**

In [6]:
from torch.utils.tensorboard import SummaryWriter
from sac.random_agent import random_agent_loop

2025-11-27 11:01:40.865414: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-27 11:01:41.506856: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-27 11:01:44.417440: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [7]:
seed = 42
num_episodes = 400


## **SAC Agent**

In [8]:
import yaml
from sac.agent import SAC


In [10]:
with open('configs/donkey_car.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Log using the Donkey Car VAE environment name
config['logger']['env_name'] = 'DonkeyVae-v0' 


### **TRAIN**

In [11]:
SAC_agent = SAC(env=env, config=config)
SAC_agent.run_training_loop(num_episodes=num_episodes, tqdm_disable=False, print_rewards=False)


NameError: name 'env' is not defined

#### **Eval trained Agent**

In [ ]:
SAC_agent.eval_agent(num_episodes=1, render_mode="human")


### **Load trained agent & Eval**

In [ ]:
# Load trained agent and run in environment
config['logger']['enabled'] = False  # Disable logging for loaded agent
SAC_agent_loaded = SAC(env=env, config=config)
# SAC_agent_loaded.load_agent('../notebooks/runs/DonkeyCar-v0/SAC/sac-donkey-car-2025_11_25-16_41_19/sac_agent.pth')


In [ ]:
SAC_agent_loaded.eval_agent(num_episodes=1, render_mode="human")


## **SB3 SAC Agent**

In [ ]:
from stable_baselines3 import SAC as SB3_SAC
from sac.utils.stable_baseline_params import get_sb3_sac_params
from sac.utils.stable_baseline_logger import RewardLoggingCallback

sb3_env_id = "DonkeyVae-v0"
sb3_params = get_sb3_sac_params(env, config, seed, env_id=sb3_env_id)

writer = SummaryWriter(f"runs/{sb3_env_id}/sac_sb3/")
sb3_sac = SB3_SAC(**sb3_params)
callback = RewardLoggingCallback(writer, num_episodes=num_episodes)

sb3_sac.learn(
    total_timesteps=num_episodes * env.spec.max_episode_steps,
    callback=callback
)

writer.close()
env.close()
